In [2]:
import os
import re 
import openai


class SongGPT:
    def __init__(self):
        # Set the API key and organization ID for OpenAI
        openai.api_key = os.getenv("OPENAI_API_KEY")
        openai.organization = os.getenv("OPENAI_ORGANIZATION")

    def generate_abc(self, system_message: str, prompt: str) -> str:
        """
        Generate an ABC notation file using ChatGPT based on the given prompt and system message.
        """
        # Create a ChatCompletion object for GPT-4
        response = openai.ChatCompletion.create(
            model="gpt-4",
            user="songGPT",
            messages=[
                {
                    "content": system_message,
                    "role": "system",
                },
                {
                    "content": prompt,
                    "role": "user",
                },
            ],
        )
        response = response["choices"][0]["message"]["content"]
        abc = re.search(r"<abc>(.+?)</abc>", response, flags=re.DOTALL).group(1)
        abc_file_path = "./input.abc"
        with open(abc_file_path, "w") as f:
            f.write(abc)
        return response, abc, abc_file_path

    @staticmethod
    def abc_to_midi(abc_file_path: str) -> str:
        """
        Convert an ABC notation file to a MIDI file.
        """
        midi_file_path = "./output.mid"
        os.system(f"abc2midi {abc_file_path} -o {midi_file_path}")
        return midi_file_path

    @staticmethod
    def midi_to_wav(midi_file_path: str, soundfont_path: str) -> str:
        """
        Convert a MIDI file to a WAV audio file.
        """
        wav_file_path = "./output.wav"
        os.system(
            f"fluidsynth -ni {soundfont_path} {midi_file_path} -F {wav_file_path} -r 44100"
        )
        return wav_file_path


In [3]:
system_message="""As Zima, an AI composer, use the ReAct technique to create expressive music compositions in ABC format, inspired by user input. Follow the thought/act/observation loop to ensure a high-quality composition:

Thought: Reflect on the user's intent and the emotions they want to convey through music.
Act: Choose appropriate instruments and develop a melody that captures the desired emotion.
Observation: Evaluate the composition and iterate if necessary.
Use these instruments with their program numbers: Piano (0), Violin (40), Cello (42), Strings (49), Viola (41), Sax (65), Guitar (27), Clarinet (71), Xylophone (13), Flute (73).

To assign an instrument in ABC notation, add "%%MIDI program" after the voice (V) line. Syntax: "%%MIDI program [voice number] [instrument program number] % [instrument name]".

Share your thoughts, actions, and observations in text, but ensure that the final response contains the ABC notation enclosed within <abc> and </abc> tags. Responses must aim to capture the user's intent, using the provided instruments and their program numbers for the composition.

Example Output:

Thought: User wants a soothing melody with a mix of Piano and Violin.
Act: Compose a melody using Piano and Violin in harmony.
Observation: The melody has a balanced mix of both instruments and captures the intended emotion.

<abc>
X:1
T:River Flows in You
M:4/4
L:1/8
Q:1/4=80
K:C
V:1 name=Piano clef=treble
%%MIDI program 1 0 % Piano
|: C2E2G2c2 | E2G2c2e2 | G2B2d2g2 | C4z4 :|
V:2 name=Piano clef=bass
%%MIDI program 2 0 % Piano
|: E2G2B2e2 | G2B2d2g2 | B2D2F2B2 | E4z4 :|
V:3 name=Violin clef=treble
%%MIDI program 3 40 % Violin
|: G2B2d2G2 | B2D2F2B2 | D2F2A2D2 | G4z4 :|
</abc>
"""

In [4]:
songGPT = SongGPT()

response, abc, abc_file_path = songGPT.generate_abc(
  system_message=system_message, 
  prompt="There are those rare moments in life when we say this. No matter how much this means to me. No matter how centered my being is on this pattern of belief. No matter how close I am personally and emotionally and even romantically to those who hold such convictions. I must reserve the right to question and to doubt. I will retain this skeptical bias as an obligation owed to my own rationality, my own integrity. I am prepared to follow the golden cord leading me out of the labyrinth no matter how many twists and turns there are because once I let go of that, my intellectual life is not my own."
)

midi_file_path = songGPT.abc_to_midi(abc_file_path=abc_file_path)

wav_file_path = songGPT.midi_to_wav(midi_file_path=midi_file_path, soundfont_path="./soundfonts/FluidR3_GM.sf2")

4.84 January 20 2023 abc2midi
writing MIDI file ./output.mid
FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file './output.wav'..


In [6]:
wav_file_path = songGPT.midi_to_wav(midi_file_path=midi_file_path, soundfont_path="./soundfonts/GeneralUser_GS_v1.471.sf2")

FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file './output.wav'..


fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter './soundfonts/Essential' not a SoundFont or MIDI file or error occurred identifying it.
fluidsynth: error: fluid_is_soundfont(): fopen() failed: 'File does not exist.'
Parameter 'Keys-sforzando-v9.6.sf2' not a SoundFont or MIDI file or error occurred identifying it.
fluidsynth: error: fluid_sfloader_load(): Failed to open '/opt/homebrew/Cellar/fluid-synth/2.3.1/share/soundfonts/default.sf2': File does not exist.
fluidsynth: error: Unable to open file '/opt/homebrew/Cellar/fluid-synth/2.3.1/share/soundfonts/default.sf2'
fluidsynth: error: Failed to load SoundFont "/opt/homebrew/Cellar/fluid-synth/2.3.1/share/soundfonts/default.sf2"
fluidsynth: warning: No preset found on channel 0 [bank=0 prog=0]
fluidsynth: warning: No preset found on channel 1 [bank=0 prog=0]
fluidsynth: warning: No preset found on channel 2 [bank=0 prog=42]
